In [1]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from url_parser import parse_url
import numpy as np

client = Client()

In [2]:
client 

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 125.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43147,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 125.60 GiB
Comm: tcp://127.0.0.1:38105,Total threads: 4
Dashboard: http://127.0.0.1:45409/status,Memory: 20.93 GiB
Nanny: tcp://127.0.0.1:42089,


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

pbar = ProgressBar()
pbar.register()

In [4]:
dat = dd.read_parquet("data/output")

In [5]:
only_us = dat[(dat.Actor1Code == "USA")]
only_china = dat[(dat.Actor1Code == "CHN")]
only_russia = dat[(dat.Actor1Code == "RUS")]

In [6]:
dat.head()

,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
GLOBALEVENTID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
963548206,20201211,202012,2020,2020.9342,USAEDU,UNITED STATES,USA,None,None,None,None,EDU,None,None,EDU,SCHOOL,None,None,None,None,None,EDU,None,None,0,030,030,03,1,4.0,2,1,2,1.01522842639594,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,20210110,https://www.ktvb.com/article/news/health/coron...
963548207,20210103,202101,2021,2021.0082,None,None,None,None,None,None,None,None,None,None,MIL,NATIONAL GUARD,None,None,None,None,None,MIL,None,None,1,036,036,03,1,4.0,70,5,70,-4.75240042017015,0,None,None,None,None,None,None,2,"California, United States",US,USCA,36.17,-119.746,CA,2,"California, United States",US,USCA,36.17,-119.746,CA,20210110,https://www.mdjonline.com/tribune/lifestyles/a...
963548213,20210103,202101,2021,2021.0082,USA,UNITED STATES,USA,None,None,None,None,None,None,None,GOV,SECRETARY OF STATE,None,None,None,None,None,GOV,None,None,0,020,020,02,1,3.0,142,38,142,-4.17371166647743,3,"Atlanta, Georgia, United States",US,USGA,33.749,-84.388,351615,3,"Atlanta, Georgia, United States",US,USGA,33.749,-84.388,351615,3,"Atlanta, Georgia, United States",US,USGA,33.749,-84.388,351615,20210110,https://accesswdun.com/article/2021/1/969753
963548214,20210103,202101,2021,2021.0082,USA,OREGON,USA,None,None,None,None,None,None,None,USAMIL,OREGON,USA,None,None,None,None,MIL,None,None,1,080,080,08,2,5.0,6,1,6,1.01522842639594,3,"Washington, District of Columbia, United States",US,USDC,38.8951,-77.0364,531871,3,"Washington, District of Columbia, United States",US,USDC,38.8951,-77.0364,531871,3,"Washington, District of Columbia, United States",US,USDC,38.8951,-77.0364,531871,20210110,https://www.ktvb.com/article/news/health/coron...
963548215,20210103,202101,2021,2021.0082,USA,UNITED STATES,USA,None,None,None,None,None,None,None,USAMIL,UNITED STATES,USA,None,None,None,None,MIL,None,None,1,080,080,08,2,5.0,4,1,4,1.01522842639594,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,2,"Oregon, United States",US,USOR,44.5672,-122.127,OR,20210110,https://www.ktvb.com/article/news/health/coron...


In [7]:
from tqdm import tqdm

events_per_feature = 10

all_features = []
country_key = []

for country, only_country in [("US",only_us), 
                              ("China",only_china), 
                              ("Russia ",only_russia)]:
    features = only_country[["EventCode", "EventBaseCode", "EventRootCode", "QuadClass", "GoldsteinScale", "AvgTone"]]
    features.dropna()
    
    composite_features = []
    arr = features.to_records().compute()
    for idx, row in enumerate(tqdm(arr)):
        float_row = []
        # we don't want the global Event ID in our feature set
        for item in list(row)[1:]:
            try:
                float_row.append(float(item))
            # we also want to ignore any rows that contain a None
            except:
                print(f"Could not convert {row}")
        composite_features.extend(float_row)
        if idx % events_per_feature == 0:
            if len(composite_features) == (len(row)-1) * events_per_feature:
                all_features.append(composite_features)
                country_key.append(country)
            composite_features = []

 15%|█████▌                                | 1135498/7734747 [00:04<00:23, 275089.51it/s]

Could not convert ('962970887', '1213', '121', '12', '3', None, '-4.98960498960499')
Could not convert ('962970936', '1213', '121', '12', '3', None, '-4.98960498960499')
Could not convert ('963003182', '1213', '121', '12', '3', None, '-5.25328330206379')


 17%|██████▍                               | 1301077/7734747 [00:04<00:24, 259246.13it/s]

Could not convert ('1023944957', '---', '---', '--', '4', None, '-5.64516129032258')
Could not convert ('1023944957', '---', '---', '--', '4', None, '-5.64516129032258')
Could not convert ('1023944957', '---', '---', '--', '4', None, '-5.64516129032258')
Could not convert ('1023944957', '---', '---', '--', '4', None, '-5.64516129032258')


 21%|████████▏                             | 1662763/7734747 [00:06<00:21, 278194.83it/s]

...

 59%|██████████████████████▉                | 733154/1245005 [00:02<00:01, 274436.41it/s]

Could not convert ('1035391866', '1213', '121', '12', '3', None, '-9.72423802612482')
Could not convert ('1035391867', '1213', '121', '12', '3', None, '-9.72423802612482')
Could not convert ('1002854353', '1213', '121', '12', '3', None, '-8.22104013514259')


 75%|█████████████████████████████          | 927567/1245005 [00:03<00:01, 173578.61it/s]

Could not convert ('1002824088', '1213', '121', '12', '3', None, '-8.09352517985611')


100%|██████████████████████████████████████| 1245005/1245005 [00:05<00:00, 248533.36it/s]


In [8]:
X = np.array(all_features)
X.shape

(986220, 60)

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
print(X[0], country_key[0])

[-0.08824913  0.16289138  0.19571414  0.14701529  0.96879678  0.92496148
 -0.08766976  0.16256776  0.19534906  0.14702525  0.96790011  0.92511737
 -0.37222614 -0.72689183 -0.693481   -0.72894741  0.75696283  0.23083749
  0.44046635  1.81407183  1.79245093  1.89505888 -1.14893226 -1.12233745
  0.43935993  1.81700666  1.79536722  1.8975552  -1.15172712 -0.51532637
 -0.29815944 -0.49448018 -0.51439737 -0.72773619  0.50041025  0.59727126
 -0.48617168 -1.08252223 -1.04872788 -0.72881091 -0.09121401 -0.64624183
 -0.47941703 -1.0632344  -1.04729799 -0.72740263 -0.11231957 -0.96066805
 -0.47868347 -1.06229561 -1.04636485 -0.72652506 -0.11205614 -0.9160337
 -0.46719299 -1.02904874 -1.04853196 -0.72911738 -0.00513855 -0.88258658] US


## K Means Clustering

In [ ]:
from sklearn.cluster import KMeans

trials = []
for i in tqdm(range(2, 20)):
    km = KMeans(n_clusters=i, 
           max_iter=300, 
           tol=0.0001, 
           random_state=42,
            verbose=1)
    km.fit(X)
    trials.append(km.inertia_)
    
plt.plot(range(2, 20), trials, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.show()

  0%|                                                             | 0/18 [00:00<?, ?it/s]/home/parvus/anaconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 82736615.89439234.
Iteration 1, inertia 55360709.611245245.
Iteration 2, inertia 53454439.284383364.
Iteration 3, inertia 53009307.35424578.
Iteration 4, inertia 52871768.25052762.
Iteration 5, inertia 52825241.08766135.
Iteration 6, inertia 52809313.82988943.
Iteration 7, inertia 52803868.6804022.
Iteration 8, inertia 52801948.67389987.
Iteration 9, inertia 52801276.48429286.
Iteration 10, inertia 52801024.332042016.
Iteration 11, inertia 52800926.428078644.
Converged at iteration 11: center shift 6.505486386612553e-05 within tolerance 9.999999999985262e-05.
Initialization complete
Iteration 0, inertia 76759500.81071112.
Iteration 1, inertia 53795721.49451295.
Iteration 2, inertia 53121427.89003311.
Iteration 3, inertia 52962604.79377222.
Iteration 4, inertia 52908475.581144065.
Iteration 5, inertia 52881658.1225184.
Iteration 6, inertia 52864528.61409597.
Iteration 7, inertia 52851870.598848484.
Iteration 8, inertia 52842234.1430481.
Itera

Iteration 21, inertia 52801813.6915598.
Iteration 22, inertia 52801597.066268764.
Iteration 23, inertia 52801430.1923376.
Iteration 24, inertia 52801311.61840223.
Iteration 25, inertia 52801222.95116609.
Converged at iteration 25: center shift 9.18778984766594e-05 within tolerance 9.999999999985262e-05.
Initialization complete
Iteration 0, inertia 87064983.78966762.
Iteration 1, inertia 53849724.11148941.
Iteration 2, inertia 53512585.17142408.
Iteration 3, inertia 53346868.05232374.
Iteration 4, inertia 53246296.3968889.
Iteration 5, inertia 53157080.529190905.
Iteration 6, inertia 53089135.110772364.
Iteration 7, inertia 53036445.195166886.
Iteration 8, inertia 52990065.61585804.
Iteration 9, inertia 52953924.27833827.
Iteration 10, inertia 52926744.34191189.
Iteration 11, inertia 52905196.75185156.
Iteration 12, inertia 52886979.494071916.
Iteration 13, inertia 52871616.33166636.
Iteration 14, inertia 52858560.96757571.
Iteration 15, inertia 52847708.12287524.
Iteration 16, inertia 

  6%|██▉                                                  | 1/18 [00:25<07:08, 25.23s/it]

Iteration 38, inertia 52801107.40433693.
Converged at iteration 38: center shift 8.464710406212538e-05 within tolerance 9.999999999985262e-05.


...

Initialization complete
Iteration 0, inertia 61117387.80066508.
Iteration 1, inertia 43305179.56064682.
Iteration 2, inertia 42173947.68745531.
Iteration 3, inertia 41519196.10756256.
Iteration 4, inertia 41203475.36045817.
Iteration 5, inertia 41064146.56050765.
Iteration 6, inertia 40949649.65112969.
Iteration 7, inertia 40802439.39353009.
Iteration 8, inertia 40691192.19660671.
Iteration 9, inertia 40631249.874149926.
Iteration 10, inertia 40599404.48975537.
Iteration 11, inertia 40580497.0604593.
Iteration 12, inertia 40568427.19845703.
Iteration 13, inertia 40559600.55816583.
Iteration 14, inertia 40552646.033186235.
Iteration 15, inertia 40546903.716729864.
Iteration 16, inertia 40542227.99138112.
Iteration 17, inertia 40538194.75780573.
Iteration 18, inertia 40534617.69533449.
Iteration 19, inertia 40531724.28406817.
Iteration 20, inertia 40529434.008350775.
Iteration 21, inertia 40527697.50414269.
Iteration 22, inertia 40526265.20295181.
Iteration 23, inertia 40525136.82487676.

Iteration 31, inertia 40315939.28786275.
Iteration 32, inertia 40315764.98664592.
Iteration 33, inertia 40315629.32492749.
Iteration 34, inertia 40315525.45000566.
Iteration 35, inertia 40315443.12268879.
Iteration 36, inertia 40315380.49002726.
Iteration 37, inertia 40315336.76597877.


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 179, in where
KeyboardInterrupt: 


Iteration 38, inertia 40315307.86392524.
Iteration 39, inertia 40315289.49604074.
Iteration 40, inertia 40315277.24640456.
Converged at iteration 40: center shift 8.479619020763598e-05 within tolerance 9.999999999985262e-05.
Initialization complete
Iteration 0, inertia 56960887.86462096.
Iteration 1, inertia 42075297.14062038.
Iteration 2, inertia 41047356.383623526.
Iteration 3, inertia 40756650.79230663.
Iteration 4, inertia 40652325.66656391.
Iteration 5, inertia 40584819.14238133.
Iteration 6, inertia 40510118.97195371.
Iteration 7, inertia 40412565.75392369.
Iteration 8, inertia 40319752.14103528.
Iteration 9, inertia 40259174.06929698.
Iteration 10, inertia 40225938.726031795.
Iteration 11, inertia 40204936.34879961.
Iteration 12, inertia 40188368.94304616.
Iteration 13, inertia 40173448.13936734.
Iteration 14, inertia 40162670.496657394.
Iteration 15, inertia 40156558.004817836.
Iteration 16, inertia 40153052.71466611.
Iteration 17, inertia 40150992.630089656.
Iteration 18, iner

In [12]:
km.labels_

array([0, 1, 1, ..., 2, 0, 0], dtype=int32)